Now, we turn to elliptic equations. Elliptic equations do not have a "time-like" variable, in the sense that the solution at every point in the domain depends on all of the boundary conditions. Picking one of the variables to designate as "time," setting up a finite difference scheme with initial conditions and boundary conditions, and stepping through "time" is unlikely to work for elliptic equations since the solution at any point depends on all of the boundary conditions, including the "future" from the perspective of our designated variable. Thus, such a scheme could never converge, as it does not depend on all of the information which the true solution depends on.

 One interpretation of elliptic equations is as steady-state versions of parabolic equations. In other words, the solution to an elliptic equation describes the equilibrium state of a system, rather than how it evolves over time. As an example, recall the forced heat equation:
$$
    u_t + \Delta u = f.
$$
Along with some initial values and boundary conditions, we can model the evolution of the system over time. It is natural to wonder if we can model what happens as time goes to infinity. Since this is a dissipative system, it is reasonable to expect that it approaches some limiting solution as time goes to infinity (contrast this to the wave equation with reflective boundary conditions, in which we would not expect things to converge as time goes to infinity). One way to impose the requirement that eventually, the solution does not change in time is to specify that $u_t = 0$. If we do this, we get the equation
$$
    \Delta u = f.
$$
This is Poisson's equation, and we have heuristically justified the interpretation of it as giving the limiting or steady-state solution to the heat equation. 

Suppose a function $u$ is the steady-state solution to some time-dependent PDE. One way to find $u$ is to begin with arbitrary initial conditions and then let it evolve until the solution changes so slowly in time that we regard it as essentially constant. This limit may be a good approximation to $u$. Numerically, this translates to iterating a numerical scheme for the time-dependent PDE, with constant boundary conditions, until the solution stabilizes.

If $U^n$ is the $n$-th iteration of the scheme, we can measure whether the solution is changing in essentially two ways: comparing $U^n$ to $U^{n+1}$ or comparing $U^n$ to $U^m$ for some $m$ many time steps in the future. The second option has more strength in detecting convergence. As a toy example, consider the series $\sum 1/k$, which diverges. Nonetheless, the term-to-term differences in the sum are just $1/k$, which goes to $0$ fairly quickly. In contrast, $\sum_{k=1}^m 1/k - \sum_{k=1}^n 1/k \approx \log(m-n)$, and we have a better chance of figuring out that we aren't getting convergence than when just looking at $1/k$. 

While the method of iterating a time-dependent version of the elliptic PDE has the advantage of applying previous knowledge to a new problem, there are better methods, such as the finite element method. For clarity, we start by developing finite element method in one dimension before extending it to higher dimensions. The one-dimensional Poisson equation  on $[0, 1]$ with forcing term $f$ and homogeneous Dirichlet boundary conditions is
\begin{cases}
    -u''(x) = f(x), x \in (0, 1),\\
    u(0) = u(1) = 0.
\end{cases}
This is just an ODE, but is not an  initial value problem due to the specification of the value of $u(1)$. If, in contrast, the boundary conditions specified $u(0)$ and $u'(0)$, we could solve this typical numerical methods for initial value problems such as Runge-Kutta.

There are two main ideas for the finite element method. The first is that we want to approximate $u$ by some function living in a finite-dimensional vector space. The second is to transform the problem into a "variational" problem on this finite-dimensional space. We start with deriving two variational forms, and then introduce the vector space.